## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread("lena.bmp", cv2.IMREAD_GRAYSCALE)

def fourier(image):
    image = image.astype(np.float32)
    F_fcn = cv2.dft(image,flags=cv2.DFT_COMPLEX_OUTPUT)
    F_fcn = np.fft.fftshift(F_fcn, [0,1])
    magnitude, phase = cv2.cartToPolar(F_fcn[:,:,0], F_fcn[:,:,1])
    magnitude_db = np.log10(1 + magnitude)
    return F_fcn, magnitude_db, phase

def show_images(img,img1,img2):
    f, ax = plt.subplots(1,3,figsize=(20,6))
    ax[0].imshow(img, 'gray')
    ax[0].axis('off')
    ax[1].imshow(img1, 'gray')
    ax[1].axis('off')
    ax[2].imshow(img2, 'gray')
    ax[2].axis('off')
    ax[0].set_title('Obraz orginalny')
    ax[1].set_title('Przefiltrowany')
    ax[2].set_title('Różnica')

def inverse_fourier(image_fft):
    image_fft = np.fft.ifftshift(image_fft, [0,1])
    image = cv2.idft(image_fft, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    image = cv2.magnitude(image[:, :, 0], image[:, :, 1])
    image = np.round(image).astype(np.uint8)
    return image

window_size = 21

hanning = np.hanning(window_size)
hanning = np.outer(hanning, hanning)

FSpaceRows = 2*np.fft.fftshift(np.fft.fftfreq(window_size))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, window_size]))
FSpaceCols = 2*np.fft.fftshift(np.fft.fftfreq(window_size))
FSpaceColsM = np.outer(np.ones([1, window_size]), FSpaceCols)
FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR >= 0.2

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3),flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

Filter_hann = hanning * FilterFI

Filtr_bor = cv2.copyMakeBorder(Filter_hann, lena.shape[0] - Filter_hann.shape[0], 0, lena.shape[1] - Filter_hann.shape[1], 0,cv2.BORDER_CONSTANT,value=0).astype(np.float32)
F_fcn_filtr = cv2.dft(Filtr_bor,flags=cv2.DFT_COMPLEX_OUTPUT)
F_fcn_filtr = np.fft.fftshift(F_fcn_filtr, [0,1])
F_fcn_filtr_abs = np.abs(F_fcn_filtr)

FSpaceRows_512 = 2*np.fft.fftshift(np.fft.fftfreq(lena.shape[0]))
FSpaceRowsM_512 = np.outer(FSpaceRows_512, np.ones([1, lena.shape[1]]))
FSpaceCols_512 = 2*np.fft.fftshift(np.fft.fftfreq(lena.shape[1]))
FSpaceColsM_512 = np.outer(np.ones([1, lena.shape[0]]), FSpaceCols_512)


figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM_512, FSpaceColsM_512, F_fcn_filtr_abs[:,:,0], rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)


lena_fft, lena_ampl, lena_phase = fourier(lena)
F_lena_filtr = lena_fft*F_fcn_filtr_abs

lena_ifft_2 = inverse_fourier(F_lena_filtr)
diff_2 = np.abs(lena-lena_ifft_2)

show_images(lena,lena_ifft_2,diff_2)
